<a href="https://colab.research.google.com/github/paulgureghian/Ludwig_Models/blob/master/Ludwig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install https://github.com/uber/ludwig/archive/master.zip -q
!pip install -U scikit-image numpy -q
!pip check

     | 17.8MB 2.0MB/s
pymc3 3.6 has requirement joblib<0.13.0, but you have joblib 0.13.2.
jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you have prompt-toolkit 1.0.15.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you have pandas 0.22.0.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.8.


In [0]:
## Connect Google Drive.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
## Set data to where the dataset is, and csv_file to where the labels csv file should be saved.

data = '/content/drive/My Drive/Vohra Wound AI/Data/Wound Dataset'
csv_file = '/content/drive/My Drive/Colab Notebooks/WoundAI/data/input/Wound Dataset/labels.csv'

In [0]:
## Create labels.csv.

import os
import csv
import skimage

def list_files(
    startpath, 
    csvfile,
    print_rows=False,
    use_full_path=False,
    filecheck=None,
):
    
    labels = csv.writer(open(csvfile, "w", newline=''))
    
    column_headers = ['Filename', 'Label']
    labels.writerow(column_headers)
    
    accepted_count = 0
    rejected_count = 0
    
    for root, dirs, files in os.walk(startpath):
        if os.path.basename(root)[0]=='.': continue # Skip hidden folders.
        for f in files:
            full_path = os.path.join(root, f)
            
            # Skip files that don't pass the filecheck.
            if filecheck is not None and not filecheck(full_path):
                rejected_count += 1
                continue
            
            label = os.path.basename(root)
            if use_full_path:
                filename = full_path
            else:
                filename = os.path.join(os.path.basename(root),f)
                
            row = [filename, label] # 
            labels.writerow(row)
            accepted_count += 1
            
            if print_rows: print(row)
    
    print('Included {} files and skipped {} files'.format(accepted_count, rejected_count))
            
def check_image(filename, extensions={'.jpg', '.png', '.gif'}, dims=None, num_channels=None, channel_index = 2):
    
    # Only allow files with the specified extensions.
    if not os.path.splitext(filename)[1] in extensions: return False 
    
    image = skimage.io.imread(filename)
    
    # Only list image files that have the allowable number of dimensions.
    if dims is not None:
        file_dims = len(image.shape)
        if file_dims != dims: return False

    # Only list image files that have the allowable number of channels.
    if num_channels is not None:
        file_num_channels = image.shape[channel_index]
        if file_num_channels != num_channels: return False
        
    return True

In [0]:
file_checker = lambda f: check_image(
    f, 
    extensions={'.jpg', '.png', '.gif'}, 
    dims=3, 
    num_channels=3, 
    channel_index = 2,
)

list_files(data, csv_file, use_full_path=True,
          filecheck = file_checker, 
          )

Included 2094 files and skipped 170 files


In [0]:
from ludwig import LudwigModel

In [0]:
## See https://uber.github.io/ludwig/user_guide/#model-definition for more information.

model_definition = {
    'input_features': [
        {
            'name': 'Filename', 'type': 'image', 'encoder': 'stacked_cnn',         
            # Image options.
            'in_memory': False,
            'resize_method': 'interpolate',
            'height': 224,
            'width': 224,
        },
    ],
    #combiner': {},
    'output_features': [
        {'name': 'Label', 'type': 'category'}
    ],
    'training': {
        'batch_size': 32,
        'epochs': 50,
    },
    'preprocessing': {
        'force_split': False,
        'split_probabilities': [0.7, 0.2, 0.1],
        'random_seed': 2398,
    },    
}

In [0]:
# train a model
model = LudwigModel(model_definition)
train_stats = model.train(data_csv=csv_file)

# or load a model
#model = LudwigModel.load(model_path)

# obtain predictions
#predictions = model.predict(test_dataframe)

#model.close()

NameError: ignored